In [87]:
import pandas as pd
import re


In [95]:
df = pd.read_csv('BC5.csv', chunksize=10000000, iterator=True)

In [96]:
df1 = df.get_chunk()

To make it iterable: 
    https://www.programmersought.com/article/2540665375/

In [97]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ProductFamily_ID    object 
 1   ProductCategory_ID  object 
 2   ProductBrand_ID     object 
 3   ProductName_ID      object 
 4   ProductPackSKU_ID   object 
 5   Point-of-Sale_ID    object 
 6   Date                object 
 7   Measures            object 
 8   Value               float64
dtypes: float64(1), object(8)
memory usage: 686.6+ MB


In [98]:
df1

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Measures,Value
0,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2017-03-04,Sell-out units,2.0
1,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2017-03-04,Sell-out values,1540.0
2,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-05-02,Sell-out units,4.0
3,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-05-02,Sell-out values,3080.0
4,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-10-24,Sell-out units,2.0
...,...,...,...,...,...,...,...,...,...
9999995,Family_1,Category_178,ProductBrand_917,ProductName_1749,ProductSKU_5299,POS_21,2017-01-25,Sell-out values,1700.0
9999996,Family_1,Category_178,ProductBrand_917,ProductName_1749,ProductSKU_5299,POS_21,2017-01-27,Sell-out values,1700.0
9999997,Family_1,Category_178,ProductBrand_917,ProductName_1749,ProductSKU_5299,POS_21,2017-01-31,Sell-out values,850.0
9999998,Family_1,Category_178,ProductBrand_917,ProductName_1749,ProductSKU_5299,POS_21,2017-07-03,Sell-out units,1.0


In [99]:
# Renaming column
df1.rename(columns={'Point-of-Sale_ID': 'POS_ID'}, inplace=True)

# Removing text from strings and keeping just numbers, downcasting to save space
df1.ProductCategory_ID = pd.to_numeric(df1.ProductCategory_ID.str.extract('(\d)', expand=False), downcast='integer')
df1.ProductFamily_ID = pd.to_numeric(df1.ProductFamily_ID.str.extract('(\d)', expand=False), downcast='integer')
df1.ProductBrand_ID = pd.to_numeric(df1.ProductBrand_ID.str.extract('(\d)', expand=False), downcast='integer')
df1.ProductName_ID = pd.to_numeric(df1.ProductName_ID.str.extract('(\d)', expand=False), downcast='integer')
df1.ProductPackSKU_ID = pd.to_numeric(df1.ProductPackSKU_ID.str.extract('(\d)', expand=False), downcast='integer')
df1.POS_ID = pd.to_numeric(df1.POS_ID.str.extract('(\d)', expand=False), downcast='integer')

# Downcasting to save space
df1.Value = pd.to_numeric(df1.Value, downcast='float')

# Convert to datetime
df1.Date = pd.to_datetime(df1.Date, infer_datetime_format=True)  

# Transform text in binary
df1.Measures.replace({'Sell-out units': 1, 'Sell-out values': 0}, inplace=True)
# Reduce its size
df1.Measures = pd.to_numeric(df1.Measures, downcast='integer')

# Rename it
df1.rename(columns={'Measures': 'isUnit'}, inplace=True)

df1

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,POS_ID,Date,isUnit,Value
0,1,1,3,6,1,1,2017-03-04,1,2.0
1,1,1,3,6,1,1,2017-03-04,0,1540.0
2,1,1,3,6,1,1,2016-05-02,1,4.0
3,1,1,3,6,1,1,2016-05-02,0,3080.0
4,1,1,3,6,1,1,2016-10-24,1,2.0
...,...,...,...,...,...,...,...,...,...
9999995,1,1,9,1,5,2,2017-01-25,0,1700.0
9999996,1,1,9,1,5,2,2017-01-27,0,1700.0
9999997,1,1,9,1,5,2,2017-01-31,0,850.0
9999998,1,1,9,1,5,2,2017-07-03,1,1.0


In [100]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ProductFamily_ID    int8          
 1   ProductCategory_ID  int8          
 2   ProductBrand_ID     int8          
 3   ProductName_ID      int8          
 4   ProductPackSKU_ID   int8          
 5   POS_ID              int8          
 6   Date                datetime64[ns]
 7   isUnit              int8          
 8   Value               float32       
dtypes: datetime64[ns](1), float32(1), int8(7)
memory usage: 181.2 MB
